In [1]:
#Import pandas, numpy and pandas
import numpy as np
import pandas as pd
import utm

In [2]:
#Read population, road and POI files

roads_csv = pd.read_csv('Input/LIM_pixels_population_roadsext_cf.csv')
poi_file = pd.read_table('Input/data_points.txt','|')

In [3]:
#Drop unnecesary fields, rename columns
poi_file = poi_file.rename(columns={'x':'lon','y':'lat'})

In [4]:
#Dictionaries
subtypes  = {
    'food':[
        '0x2a00',
        '0x2a01',
        '0x2a02',
        '0x2a03',
        '0x2a04',
        '0x2a05',
        '0x2a06',
        '0x2a07',
        '0x2a08',
        '0x2a09',
        '0x2a0a',
        '0x2a0b',
        '0x2a0c',
        '0x2a0e',
        '0x2a0f',
        '0x2a10',
        '0x2a11',
        '0x2a12',
        '0x2a13',
        '0x2a1f',
        '0x2a18',
        '0x2a1b',
        '0x2a20',
        '0x2a21',
        '0x2a22',
        '0x2a23',
        '0x2a24',
        '0x2a25',
        '0x2a26',
        '0x2a27',
        '0x2a28',
        '0x2a29',
        '0x2a2a',
        '0x2a2b',
        '0x2a2c',
        '0x2a2d',
        '0x2a2e',
        '0x2a2f'
    ],
    'accomodation':[
        '0x2b00',
        '0x2b01',
        '0x2b02',
        '0x2b03',
        '0x2b04',
        '0x2b05',
        '0x2b1f',
        '0x2b20',
        '0x2b22',
        '0x2b23',
        '0x2b24',
        '0x2b25',
        '0x2b2a',
        '0x2b2b',
        '0x2b2c',
        '0x2b07',
        '0x2b2f'
    ],
    'recreation':[
        '0x2c09',
        '0x2c29',
        '0x2c18',
        '0x2c19',
        '0x2c1a',
        '0x2c08',
        '0x2c28',
        '0x2d02',
        '0x2d0d',
        '0x2d0e',
        '0x2d22',
        '0x2d2d',
        '0x2d2e',
        '0x4600',
        '0x462f'
    ],
    'shopping':[
        '0x2e00',
        '0x2e01',
        '0x2e02',
        '0x2e03',
        '0x2e04',
        '0x2e05',
        '0x2e06',
        '0x2e07',
        '0x2e08',
        '0x2e09',
        '0x2e0a',
        '0x2e0b',
        '0x2e0c',
        '0x2e0f',
        '0x2e10',
        '0x2e1f',
        '0x2e20',
        '0x2e21',
        '0x2e22',
        '0x2e23',
        '0x2e24',
        '0x2e25',
        '0x2e26',
        '0x2e27',
        '0x2e28',
        '0x2e29',
        '0x2e2a',
        '0x2e2b',
        '0x2e2c',
        '0x2e2f',
    ],
    'services':[
        '0x2e06',
        '0x2f01',
        '0x2f16',
        '0x2f02',
        '0x2f22',
        '0x2f07',
        '0x2f0a',
        '0x2f0b',
        '0x2f0c',
        '0x2f0d',
        '0x2f0e',
        '0x2f22',
        '0x2f23',
        '0x2f27',
        '0x2f2a',
        '0x2f2b',
        '0x2f2c',
        '0x2f2d',
        '0x2f2e',
        '0x3100',
        '0x3116',
        '0x3117',
        '0x3118',
        '0x311a',
        '0x311b',
        '0x311c',
        '0x311d',
        '0x311e',
        '0x311f',
        '0x3120',
        '0x312f',
        '0x4c00',
        '0x4c2f',
        '0x4d00',
        '0x4d2f',
        '0x4e00',
        '0x4e2f',
        '0x5100',
        '0x512f',
        '0x2f23',
        '0x2f0e',
        '0x2f2e',
        '0x2f05',
        '0x2f01',
        '0x640f',
        '0x642f',
        '0x2f06',
        '0x2f04',
        '0x2f24',
        '0x5900',
        '0x592f',
        '0x2f08',
        '0x2f1f',
        '0x2f28',
        '0x2f1d',
        '0x2f1e',
        '0x2f26',
        '0x2f0b',
        '0x2f04',
        '0x2f18',
        '0x2f2b',
        '0x4d00',
        '0x4d2f',
        '0x2f11',
        '0x2f12',
    ]
}

In [5]:
food_dataframe = pd.DataFrame(subtypes['food'],columns=['GARMINCODE'])
accomodation_dataframe = pd.DataFrame(subtypes['accomodation'],columns=['GARMINCODE'])
recreation_dataframe = pd.DataFrame(subtypes['recreation'],columns=['GARMINCODE'])
shopping_dataframe = pd.DataFrame(subtypes['shopping'],columns=['GARMINCODE'])
services_dataframe = pd.DataFrame(subtypes['services'],columns=['GARMINCODE'])

In [6]:
#Apply UTM
for index, row in poi_file.iterrows():
    [east, north, zone_n, zone_l] = utm.from_latlon(row['lat'], row['lon'])
    poi_file.ix[index,'utm_n'] = north
    poi_file.ix[index, 'utm_e'] = east

In [7]:
#Create arrays for the new columns

counts = []
food = []
accom = []
recreat = []
serv = []
shop = []
foodserv = []
food_garmins = []
accom_garmins = []
recreat_garmins = []
serv_garmins = []
shop_garmins = []

In [9]:
#For loop with dictionary

for row in roads_csv.iterrows():
    utm_n = row[1].loc['utm_n']
    utm_e = row[1].loc['utm_e']
    frame = poi_file[(poi_file['utm_n'] <= (utm_n + 500))
                    & (poi_file['utm_n'] >= (utm_n - 500))
                    & (poi_file['utm_e'] <= (utm_e + 500))
                    & (poi_file['utm_e'] >= (utm_e - 500))]
    if frame.shape[0] != 0:
        counts.append(frame.shape[0])
        #Join frame with the corresponding subtypes
        frame_food = pd.merge(frame,food_dataframe,how='inner',on='GARMINCODE')
        frame_accomodation = pd.merge(frame,accomodation_dataframe,how='inner',on='GARMINCODE')
        frame_services = pd.merge(frame,services_dataframe,how='inner',on='GARMINCODE')
        frame_shopping = pd.merge(frame,shopping_dataframe,how='inner',on='GARMINCODE')
        frame_recreation = pd.merge(frame,recreation_dataframe,how='inner',on='GARMINCODE')
        
        
        foodserv.append(frame_food.shape[0]+frame_accomodation.shape[0]+frame_recreation.shape[0])
        
        if frame_food.shape[0] != 0:
            food_garmins.append(';'.join(frame_food['GARMINCODE']))
            food.append(frame_food.shape[0])
        else:
            food_garmins.append('')
            food.append(0)
        
        if frame_accomodation.shape[0] != 0:
            accom_garmins.append(';'.join(frame_accomodation['GARMINCODE']))
            accom.append(frame_accomodation.shape[0])
        else:
            accom_garmins.append('')
            accom.append(0)
        
        if frame_services.shape[0] != 0:
            serv_garmins.append(';'.join(frame_services['GARMINCODE']))
            serv.append(frame_services.shape[0])
        else:
            serv_garmins.append('')
            serv.append(0)
        
        if frame_recreation.shape[0] != 0:
            recreat_garmins.append(';'.join(frame_recreation['GARMINCODE']))
            recreat.append(frame_recreation.shape[0])
        else:
            recreat_garmins.append('')
            recreat.append(0)
        
        if frame_shopping.shape[0] != 0:
            shop_garmins.append(';'.join(frame_shopping['GARMINCODE']))
            shop.append(frame_shopping.shape[0])
        else:
            shop_garmins.append('')
            shop.append(0)
    else:
        foodserv.append(0)
        counts.append(0)
        food.append(0)
        accom.append(0)
        recreat.append(0)
        serv.append(0)
        shop.append(0)
        shop_garmins.append('')
        food_garmins.append('')
        accom_garmins.append('')
        recreat_garmins.append('')
        serv_garmins.append('') 

In [10]:
#Convert arrays to Pandas Series
counts_series = pd.Series(counts)
food_series = pd.Series(food)
accomodation_series = pd.Series(accom)
recreation_series = pd.Series(recreat)
services_series = pd.Series(serv)
shopping_series = pd.Series(shop)
food_service_series = pd.Series(foodserv)
food_series_garmins = pd.Series(food_garmins)
accomodation_series_garmins = pd.Series(accom_garmins)
recreation_series_garmins = pd.Series(recreat_garmins)
services_series_garmins = pd.Series(serv_garmins)
shopping_series_garmins = pd.Series(shop_garmins)

In [11]:
roads_csv['POI_count'] = counts_series
roads_csv['Food and Drink count'] = food_series
roads_csv['Accomodation count'] = accomodation_series 
roads_csv['Recreation count'] = recreation_series
roads_csv['Services count'] = services_series
roads_csv['Shopping count'] = shopping_series
roads_csv['Food and Drink Garmins'] = food_series_garmins
roads_csv['Accomodation Garmins'] = accomodation_series_garmins
roads_csv['Recreation Garmins'] = recreation_series_garmins
roads_csv['Services Garmins'] = services_series_garmins
roads_csv['Shopping Gramins'] = shopping_series_garmins
roads_csv['Food Service'] = food_service_series

In [12]:
roads_csv.to_csv('Output/economicOutput_rev.csv')

End of code

In [13]:
poi_file[(poi_file['utm_n'] <= (8637276.063 + 500))
                    & (poi_file['utm_n'] >= (8637276.063 - 500))
                    & (poi_file['utm_e'] <= (301380.8033 + 500))
                    & (poi_file['utm_e'] >= (301380.8033 - 500))]

,FID,lon,lat,NOMBRE,GARMINCODE,GIROTIPO,TELEFONO,NOMBRE_VIA,CUADRA,URBANIZACI,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMEN,PAIS,LONGITUDE,LATITUDE,TOT,utm_n,utm_e
1495,1495,-76.827747,-12.322466,Club Mediterraneo,0x2c09,club,,CARR PAMANERICANA SUR,,UB JOSE OLAYA,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.827747,-12.322466,"-76.8277, -12.3225",8.637112e+06,301241.905838
3670,3670,-76.830793,-12.316765,Km 38,0x5a00,locale,,CARR PANAMERICANA SUR,,,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.830793,-12.316765,"-76.8308, -12.3168",8.637740e+06,300906.264362
3671,3671,-76.825867,-12.324617,Km 39,0x5a00,locale,,CARR PANAMERICANA SUR,,,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.825867,-12.324617,"-76.8259, -12.3246",8.636875e+06,301448.028665
19720,19720,-76.829692,-12.317579,Almacenes del Peru,0x2e04,,,CARR PANAMERICANA SUR,,,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.829692,-12.317579,"-76.8297, -12.3176",8.637651e+06,301026.646062
19722,19722,-76.827257,-12.323645,Praia,0x2d02,discoteca,,CARR ANTIGUA PANAMERICANA SUR,,UB JOSE OLAYA,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.827257,-12.323645,"-76.8273, -12.3236",8.636981e+06,301296.095462
19723,19723,-76.827339,-12.323527,Hostal el Coral,0x2b01,,,CARR ANTIGUA PANAMERICANA SUR,,UB JOSE OLAYA,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.827339,-12.323527,"-76.8273, -12.3235",8.636994e+06,301287.086763
19724,19724,-76.827783,-12.325371,Club Cafae SE,0x2c08,,,,,,,PUNTA HERMOSA,LIMA,LIMA,PERU,-76.827783,-12.325371,"-76.8278, -12.3254",8.636790e+06,301240.178838


In [19]:
roads_csv[roads_csv['Food Service'] != 0]

,pixel_ID,lat,lon,utm_n,utm_e,population,count_intersections,street_length_avg,street_length_total,streets_per_node_avg,...,Accomodation count,Recreation count,Services count,Shopping count,Food and Drink Garmins,Accomodation Garmins,Recreation Garmins,Services Garmins,Shopping Gramins,Food Service
51,51,-12.339065,-76.826587,8635276.063,301380.8033,29,65,56.889811,5973.430206,3.188406,...,0,0,0,2,0x2a0b;0x2a00;0x2a00;0x2a00,,,,0x2e04;0x2e05,4
117,117,-12.330026,-76.826524,8636276.063,301380.8033,1461,114,77.984326,13725.241410,3.041322,...,2,1,4,10,0x2a00;0x2a00;0x2a00;0x2a00;0x2a00;0x2a00;0x2a...,0x2b01;0x2b01,0x2d02,0x2f08;0x2f06;0x2f06;0x2f06,0x2e02;0x2e02;0x2e02;0x2e02;0x2e02;0x2e02;0x2e...,24
182,182,-12.320926,-76.835654,8637276.063,300380.8033,474,16,80.515030,1851.845693,2.833333,...,2,0,1,0,0x2a00;0x2a00;0x2a00;0x2a00;0x2a00;0x2a00;0x2a...,0x2b01;0x2b01,,0x2f06,,12
183,183,-12.320988,-76.826461,8637276.063,301380.8033,222,3,30.598471,91.795413,3.000000,...,1,3,0,1,,0x2b01,0x2c09;0x2d02;0x2c08,,0x2e04,4
247,247,-12.311826,-76.844783,8638276.063,299380.8033,1309,29,83.069127,3156.626833,3.172414,...,1,0,0,1,,0x2b01,,,0x2e02,1
248,248,-12.311887,-76.835591,8638276.063,300380.8033,3558,57,85.523771,7697.139372,3.030769,...,5,2,5,15,0x2a04;0x2a0b;0x2a0b;0x2a0b;0x2a0b;0x2a0b;0x2a...,0x2b01;0x2b01;0x2b01;0x2b01;0x2b01,0x2d02;0x2d02,0x2f01;0x2f01;0x2f06;0x2f06;0x2f11,0x2e02;0x2e02;0x2e02;0x2e02;0x2e02;0x2e02;0x2e...,28
312,312,-12.302725,-76.853912,8639276.063,298380.8033,1768,63,60.119951,6192.354978,3.260870,...,0,0,0,3,0x2a0b,,,,0x2e02;0x2e02;0x2e02,1
313,313,-12.302787,-76.844720,8639276.063,299380.8033,6137,96,95.151978,13701.884890,3.140000,...,0,0,2,0,0x2a00,,,0x2f01;0x2f01,,1
314,314,-12.302849,-76.835528,8639276.063,300380.8033,536,0,0.000000,0.000000,0.000000,...,1,1,0,1,0x2a00,0x2b01,0x2d02,,0x2e02,3
377,377,-12.293624,-76.863040,8640276.063,297380.8033,380,17,82.914264,1658.285272,3.117647,...,0,1,2,0,,,0x2c08,0x2f01;0x2f01,,1
